In [1]:
# Use server from examples/servers/streamable-http-stateless/
from dotenv import load_dotenv
from langgraph.checkpoint.memory import MemorySaver
from langgraph.prebuilt import create_react_agent
from langchain_core.messages import HumanMessage
from langchain_core.runnables import RunnableConfig
from langchain_tavily import TavilySearch

load_dotenv()

True

In [ ]:
# pyright: strict = false
# type: ignore
async def call_search_agent(config: RunnableConfig, memory: MemorySaver) -> None:
    """Run a class todo agent using the react agent framework and a remote MCP server."""
    search_tool = TavilySearch(max_results=5)
    tools = [search_tool]
    graph = create_react_agent(
        "groq:llama-3.3-70b-versatile", tools, checkpointer=memory)

    # Create event stream
    events = graph.astream_events({
        "messages": [
            HumanMessage(
                content="What is the capital of Ethiopia?",
            ),
        ]
    }, config=config, version="v2")

    # Handle stream output
    async for event in events:
        if event["event"] == "on_chat_model_stream":
            if "chunk" in event["data"]:
                print(event["data"]["chunk"].content, end="", flush=True)

In [7]:
shared_memory = MemorySaver()
 # Define thread config
thread_config = RunnableConfig(
    {"configurable": {
        "thread_id": 18,
    }}
)

await call_search_agent(config=thread_config, memory=shared_memory)

The capital of Ethiopia is Addis Ababa.